In [1]:
import pandas as pd

# Load the dataset from the parquet file
# This might take a few seconds as the file is large
df = pd.read_parquet('data/yellow_tripdata_2024-01.parquet')

# Display a summary of the dataframe's columns, data types, and non-null values
print("--- DataFrame Info ---")
df.info()

# Display the first 5 rows of the dataframe to see what the data looks like
print("\n--- First 5 Rows ---")
df.head()

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2964624 entries, 0 to 2964623
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18 

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.0


In [3]:
# --- Data Cleaning ---

# 1. Convert pickup and dropoff times to datetime objects, just in case they aren't already.
#    This is good practice although .info() showed they were correct.
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# 2. Filter out trips with no passengers or zero trip distance.
#    These are likely data errors or invalid trips.
df_filtered = df[(df['passenger_count'] > 0) & (df['trip_distance'] > 0)].copy()

# 3. Remove outliers. We'll consider trips between 0.1 and 100 miles as valid.
df_filtered = df_filtered[(df_filtered['trip_distance'] >= 0.1) & (df_filtered['trip_distance'] <= 100)]

# 4. Select only the columns we need for demand prediction.
df_final = df_filtered[['tpep_pickup_datetime', 'PULocationID']].copy()


# --- Verification ---

# Print the shape of the original and cleaned dataframes to see how many rows we removed.
print(f"Original dataframe shape: {df.shape}")
print(f"Cleaned dataframe shape: {df_final.shape}")

# Display the first 5 rows of our final, clean dataframe.
print("\n--- First 5 Rows of Cleaned Data ---")
df_final.head()

Original dataframe shape: (2964624, 19)
Cleaned dataframe shape: (2747075, 2)

--- First 5 Rows of Cleaned Data ---


,tpep_pickup_datetime,PULocationID
0,2024-01-01 00:57:55,186
1,2024-01-01 00:03:00,140
2,2024-01-01 00:17:06,236
3,2024-01-01 00:36:38,79
4,2024-01-01 00:46:51,211


In [4]:
# --- CORRECTED Step 5: Feature Engineering ---

# Create a fresh copy of our clean data to work with.
data_for_engineering = df_final.copy()

# IMPORTANT FIX: Set the datetime column as the index before grouping.
# This is the standard and most reliable way to perform time-based operations in pandas.
data_for_engineering.set_index('tpep_pickup_datetime', inplace=True)

# 1. Group by the hourly frequency on the new index and by the location ID column.
demand_df = data_for_engineering.groupby([
    pd.Grouper(freq='h'), 
    'PULocationID'
]).size().reset_index(name='demand')

# 2. Create the same time-based features from the timestamp column.
demand_df['hour'] = demand_df['tpep_pickup_datetime'].dt.hour
demand_df['day_of_week'] = demand_df['tpep_pickup_datetime'].dt.dayofweek  # Monday=0, Sunday=6
demand_df['day_of_month'] = demand_df['tpep_pickup_datetime'].dt.day
demand_df['month'] = demand_df['tpep_pickup_datetime'].dt.month


# --- Verification ---
print("--- Corrected Feature Engineered DataFrame ---")
print("Note: The dates should now all be in January 2024.")

# Display the first 10 rows to verify the fix.
demand_df.head(10)

--- Corrected Feature Engineered DataFrame ---
Note: The dates should now all be in January 2024.


,tpep_pickup_datetime,PULocationID,demand,hour,day_of_week,day_of_month,month
0,2002-12-31 22:00:00,170,2,22,1,31,12
1,2009-01-01 00:00:00,138,1,0,3,1,1
2,2009-01-01 23:00:00,137,1,23,3,1,1
3,2009-01-01 23:00:00,237,1,23,3,1,1
4,2023-12-31 23:00:00,68,1,23,6,31,12
5,2023-12-31 23:00:00,90,1,23,6,31,12
6,2023-12-31 23:00:00,138,1,23,6,31,12
7,2023-12-31 23:00:00,144,1,23,6,31,12
8,2023-12-31 23:00:00,161,1,23,6,31,12
9,2023-12-31 23:00:00,163,1,23,6,31,12


In [5]:
# --- Diagnostic Step: Check the Date Range ---

# Let's inspect the minimum and maximum dates in our cleaned data
# before we do any grouping. This will tell us if there are bad dates.

min_date = df_final['tpep_pickup_datetime'].min()
max_date = df_final['tpep_pickup_datetime'].max()

print(f"Earliest pickup date found: {min_date}")
print(f"Latest pickup date found: {max_date}")

Earliest pickup date found: 2002-12-31 22:59:39
Latest pickup date found: 2024-02-01 00:01:15


In [6]:
# --- FINAL & ROBUST Data Cleaning and Feature Engineering ---

# 1. Start from our filtered dataframe (after removing 0 passengers, etc.)
final_df = df_filtered.copy()

# 2. THE FIX: Explicitly filter to keep only data where the year is 2024.
#    This is a guaranteed way to remove all rows with corrupted dates.
final_df = final_df[final_df['tpep_pickup_datetime'].dt.year == 2024]

# 3. Now select only the two columns we need for our model
final_df = final_df[['tpep_pickup_datetime', 'PULocationID']]

# 4. Set the index for robust grouping
final_df.set_index('tpep_pickup_datetime', inplace=True)

# 5. Group by hour and location to calculate demand
demand_df = final_df.groupby([
    pd.Grouper(freq='h'),
    'PULocationID'
]).size().reset_index(name='demand')

# 6. Create our time-based features
demand_df['hour'] = demand_df['tpep_pickup_datetime'].dt.hour
demand_df['day_of_week'] = demand_df['tpep_pickup_datetime'].dt.dayofweek
demand_df['day_of_month'] = demand_df['tpep_pickup_datetime'].dt.day
demand_df['month'] = demand_df['tpep_pickup_datetime'].dt.month

# --- Verification ---
print(f"Shape after filtering for year 2024: {final_df.shape}")
print("\n--- Final, Corrected Feature Engineered DataFrame ---")
demand_df.head(10)

Shape after filtering for year 2024: (2747060, 1)

--- Final, Corrected Feature Engineered DataFrame ---


,tpep_pickup_datetime,PULocationID,demand,hour,day_of_week,day_of_month,month
0,2024-01-01,4,21,0,0,1,1
1,2024-01-01,7,1,0,0,1,1
2,2024-01-01,12,3,0,0,1,1
3,2024-01-01,13,11,0,0,1,1
4,2024-01-01,14,1,0,0,1,1
5,2024-01-01,18,1,0,0,1,1
6,2024-01-01,20,1,0,0,1,1
7,2024-01-01,24,18,0,0,1,1
8,2024-01-01,25,1,0,0,1,1
9,2024-01-01,33,2,0,0,1,1


In [ ]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import joblib
from xgboost.callback import EarlyStopping # <-- ADD THIS NEW IMPORT

# --- Part 1: Splitting the Data ---
X = demand_df.drop(['tpep_pickup_datetime', 'demand'], axis=1)
y = demand_df['demand']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Testing set shape: {X_test.shape}")


# --- Part 2: Training the XGBoost Model ---
print("\n--- Training XGBoost Model ---")

xgbr = xgb.XGBRegressor(objective='reg:squarederror',
                        n_estimators=1000,
                        learning_rate=0.05,
                        max_depth=6,
                        subsample=0.8,
                        colsample_bytree=0.8,
                        random_state=42,
                        eval_metric='rmse',
                        n_jobs=-1)

# --- THE FIX for XGBoost 2.0+ ---
# Early stopping is now handled via a 'callback'.
# We create an EarlyStopping object to monitor the performance and stop if it doesn't improve.
early_stop = EarlyStopping(rounds=50, save_best=True)

# Train the model using the new 'callbacks' argument
xgbr.fit(X_train, y_train,
         eval_set=[(X_test, y_test)],
         callbacks=[early_stop], # <-- USE THE NEW CALLBACKS ARGUMENT
         verbose=True)


# --- Part 3: Saving the Trained Model ---
joblib.dump(xgbr, 'demand_predictor_model.joblib')
print("\n--- Model training complete and saved as demand_predictor_model.joblib ---")

In [ ]:
import xgboost
print(xgboost.__version__)

In [7]:
# Run this cell to retrain and save the compatible model
from sklearn.model_selection import train_test_split
import xgboost as xgb
import joblib

X = demand_df.drop(['tpep_pickup_datetime', 'demand'], axis=1)
y = demand_df['demand']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgbr = xgb.XGBRegressor(objective='reg:squarederror',
                        eval_metric='rmse',
                        n_estimators=1000,
                        learning_rate=0.05,
                        max_depth=6,
                        subsample=0.8,
                        colsample_bytree=0.8,
                        random_state=42,
                        n_jobs=-1)

xgbr.fit(X_train, y_train,
         eval_set=[(X_test, y_test)],
         early_stopping_rounds=50,
         verbose=True)

joblib.dump(xgbr, 'demand_predictor_model.joblib')
print("\n--- Model re-trained and saved with xgboost v1.7.6 ---")

[0]	validation_0-rmse:78.04914
[1]	validation_0-rmse:75.96366
[2]	validation_0-rmse:74.03647
[3]	validation_0-rmse:72.23253
[4]	validation_0-rmse:70.51512
[5]	validation_0-rmse:69.01605
[6]	validation_0-rmse:67.40821
[7]	validation_0-rmse:65.84949


C:\Users\mohan\Downloads\projects\dynamic-pricing-engine\venv\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[8]	validation_0-rmse:64.40721
[9]	validation_0-rmse:63.06082
[10]	validation_0-rmse:62.44328
[11]	validation_0-rmse:61.11254
[12]	validation_0-rmse:59.89537
[13]	validation_0-rmse:58.75390
[14]	validation_0-rmse:57.70093
[15]	validation_0-rmse:56.65868
[16]	validation_0-rmse:55.81822
[17]	validation_0-rmse:55.43873
[18]	validation_0-rmse:55.08922
[19]	validation_0-rmse:54.26828
[20]	validation_0-rmse:53.96866
[21]	validation_0-rmse:53.02027
[22]	validation_0-rmse:52.30163
[23]	validation_0-rmse:52.05744
[24]	validation_0-rmse:51.83322
[25]	validation_0-rmse:51.05590
[26]	validation_0-rmse:50.44551
[27]	validation_0-rmse:49.75976
[28]	validation_0-rmse:49.12552
[29]	validation_0-rmse:48.48100
[30]	validation_0-rmse:47.90556
[31]	validation_0-rmse:47.33375
[32]	validation_0-rmse:46.61405
[33]	validation_0-rmse:46.16951
[34]	validation_0-rmse:45.66375
[35]	validation_0-rmse:45.06512
[36]	validation_0-rmse:44.61583
[37]	validation_0-rmse:44.22775
[38]	validation_0-rmse:43.87210
[39]	valid